In [1]:
# Author: Rangeesh Muthaiyan

from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Sequential
from keras import layers
from keras import callbacks
from tensorflow.examples.tutorials.mnist import input_data
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels
import time

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training_verbosity = 2
# Download data if needed and import.
mnist = input_data.read_data_sets('tempData', one_hot=True, reshape=False)
val_images = mnist.validation.images
val_labels = mnist.validation.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use tf.data to implement this functionality.


...

Instructions for updating:
Please use tf.data to implement this functionality.


Extracting tempData\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.


Instructions for updating:
Please use tf.one_hot on tensors.


Extracting tempData\t10k-images-idx3-ubyte.gz
Extracting tempData\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Create LeNet model
model = Sequential()
model.add(Conv2D(6,
                 [5, 5],
                 input_shape=[28, 28, 1],
                 activation='relu',
                 name='conv_1'))
model.add(MaxPool2D())
model.add(Conv2D(16, [5, 5], activation='relu', name='conv_2'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(120, activation='relu', name='dense_1'))
model.add(Dense(84, activation='relu', name='dense_2'))
model.add(Dense(10, activation='softmax', name='dense_3'))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         min_delta=0,
                                         patience=10,
                                         verbose=training_verbosity,
                                         mode='auto')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.1,
                                        patience=5,
                                        verbose=training_verbosity,
                                        mode='auto',
                                        epsilon=0.0001,
                                        cooldown=0,
                                        min_lr=0)


C:\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train LeNet on MNIST
results = model.fit(mnist.train.images,
                    mnist.train.labels,
                    epochs=5,
                    batch_size=128,
                    verbose=2,
                    validation_data=(val_images, val_labels),
                    callbacks=[early_stopping, reduce_lr])

loss,accuracy_original = model.evaluate(val_images, val_labels, batch_size=128, verbose=2)

model.save("./LeNet5_test/before_pruning.h5")
print('original model accuracy:', accuracy_original, '\n')

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 55000 samples, validate on 5000 samples
Epoch 1/5
 - 10s - loss: 0.3334 - acc: 0.9016 - val_loss: 0.1098 - val_acc: 0.9672
Epoch 2/5
 - 10s - loss: 0.0883 - acc: 0.9727 - val_loss: 0.0653 - val_acc: 0.9814
Epoch 3/5
 - 10s - loss: 0.0603 - acc: 0.9807 - val_loss: 0.0608 - val_acc: 0.9832
Epoch 4/5
 - 10s - loss: 0.0475 - acc: 0.9853 - val_loss: 0.0510 - val_acc: 0.9856
Epoch 5/5
 - 10s - loss: 0.0394 - acc: 0.9879 - val_loss: 0.0421 - val_acc: 0.9888
original model accuracy: 0.9888 



In [6]:
t = time.clock()
l,a = model.evaluate(val_images, val_labels, batch_size=128, verbose=2)
t = time.clock() - t
print(f"Test Accuracy: {a:.4f}, Timing: {t:.2f}s")

Test Accuracy: 0.9888, Timing: 0.48s


In [7]:
layer_name = 'dense_1'

accuracy = accuracy_original

while accuracy > (0.98):
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, val_images)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)

    model.save("./LeNet5_mnist_results/after_pruning.h5")
    print('model accuracy after pruning: ', accuracy, '\n')


    results = model.fit(mnist.train.images,
                        mnist.train.labels,
                        epochs=5,
                        batch_size=128,
                        callbacks=[early_stopping, reduce_lr])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)
    model.save("./LeNet5_mnist_results/after_pruning_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')
        

Deleting 23/120 channels from layer: dense_1
5000/5000 [==============================] - 1s 106us/step
model accuracy after pruning:  0.9856 

Epoch 1/5
55000/55000 [==============================] - 10s 181us/step - loss: 0.0378 - acc: 0.9879
Epoch 2/5
  896/55000 [..............................] - ETA: 9s - loss: 0.0320 - acc: 0.9888

C:\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Anaconda\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


55000/55000 [==============================] - 10s 177us/step - loss: 0.0313 - acc: 0.9901
Epoch 3/5
55000/55000 [==============================] - 10s 175us/step - loss: 0.0265 - acc: 0.9916
Epoch 4/5
55000/55000 [==============================] - 10s 175us/step - loss: 0.0221 - acc: 0.9927
Epoch 5/5
5000/5000 [==============================] - 0s 95us/step
model accuracy after retraining:  0.9886 

Deleting 19/97 channels from layer: dense_1
5000/5000 [==============================] - 1s 111us/step
model accuracy after pruning:  0.9662 

Epoch 1/5
55000/55000 [==============================] - 10s 182us/step - loss: 0.0254 - acc: 0.9912
Epoch 2/5
55000/55000 [==============================] - 10s 174us/step - loss: 0.0193 - acc: 0.9934
Epoch 3/5
55000/55000 [==============================] - 10s 175us/step - loss: 0.0176 - acc: 0.9938
Epoch 4/5
55000/55000 [==============================] - 10s 174us/step - loss: 0.0130 - acc: 0.9960
Epoch 5/5
5000/5000 [============================

In [8]:
t = time.clock()
l,a = model.evaluate(val_images, val_labels, batch_size=128, verbose=2)
t = time.clock() - t
print(f"Test Accuracy: {a:.4f}, Timing: {t:.2f}s")

Test Accuracy: 0.9768, Timing: 0.50s


In [9]:
layer_name = 'dense_2'

while accuracy > (0.97):
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, val_images)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)

    model.save("./LeNet5_mnist_results/after_pruning.h5")
    print('model accuracy after pruning: ', accuracy, '\n')


    results = model.fit(mnist.train.images,
                        mnist.train.labels,
                        epochs=5,
                        batch_size=128,
                        callbacks=[early_stopping, reduce_lr])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)
    model.save("./LeNet5_mnist_results/after_pruning_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

Deleting 13/84 channels from layer: dense_2
5000/5000 [==============================] - 1s 252us/step
model accuracy after pruning:  0.9744 

Epoch 1/5
55000/55000 [==============================] - 11s 201us/step - loss: 0.0507 - acc: 0.9834
Epoch 2/5
  768/55000 [..............................] - ETA: 10s - loss: 0.0354 - acc: 0.9909

C:\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Anaconda\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


55000/55000 [==============================] - 11s 192us/step - loss: 0.0464 - acc: 0.9844
Epoch 3/5
55000/55000 [==============================] - 11s 191us/step - loss: 0.0429 - acc: 0.9860
Epoch 4/5
55000/55000 [==============================] - 11s 194us/step - loss: 0.0414 - acc: 0.9864
Epoch 5/5
5000/5000 [==============================] - 0s 99us/step
model accuracy after retraining:  0.978 

Deleting 17/71 channels from layer: dense_2
5000/5000 [==============================] - 1s 259us/step
model accuracy after pruning:  0.8656 

Epoch 1/5
55000/55000 [==============================] - 12s 212us/step - loss: 0.0451 - acc: 0.9856
Epoch 2/5
55000/55000 [==============================] - 11s 195us/step - loss: 0.0380 - acc: 0.9876
Epoch 3/5
55000/55000 [==============================] - 11s 195us/step - loss: 0.0359 - acc: 0.9884
Epoch 4/5
55000/55000 [==============================] - 11s 194us/step - loss: 0.0349 - acc: 0.9886
Epoch 5/5
5000/5000 [=============================

In [10]:
t = time.clock()
l,a = model.evaluate(val_images, val_labels, batch_size=128, verbose=2)
t = time.clock() - t
print(f"Test Accuracy: {a:.4f}, Timing: {t:.2f}s")

Test Accuracy: 0.9566, Timing: 0.51s


In [13]:
layer_name = 'conv_2'

while accuracy > (0.95):
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, val_images)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)

    model.save("./LeNet5_mnist_results/after_pruning_conv2.h5")
    print('model accuracy after pruning: ', accuracy, '\n')


    results = model.fit(mnist.train.images,
                        mnist.train.labels,
                        epochs=5,
                        batch_size=128,
                        callbacks=[early_stopping, reduce_lr])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)
    model.save("./LeNet5_mnist_results/after_pruning_conv2_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

Deleting 3/16 channels from layer: conv_2
5000/5000 [==============================] - 2s 391us/step
model accuracy after pruning:  0.9138 

Epoch 1/5
55000/55000 [==============================] - 12s 218us/step - loss: 0.1900 - acc: 0.9528
Epoch 2/5
  896/55000 [..............................] - ETA: 10s - loss: 0.1555 - acc: 0.9643

C:\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Anaconda\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


55000/55000 [==============================] - 10s 186us/step - loss: 0.1525 - acc: 0.9625
Epoch 3/5
55000/55000 [==============================] - 11s 196us/step - loss: 0.1345 - acc: 0.9654
Epoch 4/5
55000/55000 [==============================] - 10s 190us/step - loss: 0.1227 - acc: 0.9675
Epoch 5/5
5000/5000 [==============================] - 1s 113us/step
model accuracy after retraining:  0.9656 

Deleting 2/13 channels from layer: conv_2
5000/5000 [==============================] - 2s 448us/step
model accuracy after pruning:  0.8108 

Epoch 1/5
55000/55000 [==============================] - 13s 243us/step - loss: 0.1847 - acc: 0.9480
Epoch 2/5
55000/55000 [==============================] - 11s 207us/step - loss: 0.1399 - acc: 0.9604
Epoch 3/5
55000/55000 [==============================] - 12s 209us/step - loss: 0.1283 - acc: 0.9628
Epoch 4/5
55000/55000 [==============================] - 12s 210us/step - loss: 0.1203 - acc: 0.9660
Epoch 5/5
5000/5000 [=============================

In [16]:
t = time.clock()
l,a = model.evaluate(val_images, val_labels, batch_size=128, verbose=2)
t = time.clock() - t
print(f"Test Accuracy: {a:.4f}, Timing: {t:.2f}s")

Test Accuracy: 0.9376, Timing: 1.21s
